In [ ]:
from datasets import load_dataset
from multiprocessing import Pool
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#from spellchecker import SpellChecker
import ftfy
import unicodedata
from pathlib import Path


# 1. Lectura y split 

In [2]:
#wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -O tinyshakespeare.txt

dataset = load_dataset("text", data_files={"raw": "./datasets/tinyshakespeare.txt"})

# Dividir en train (90%) y test (10%)
train_test = dataset["raw"].train_test_split(test_size=0.1, seed=42)

# Dividir train en train (90%) y validation (10%)
train_valid = train_test["train"].train_test_split(test_size=0.1, seed=42)

# Reunir en un DatasetDict
tinishakespeare = {
    "train": train_valid["train"],
    "validation": train_valid["test"],
    "test": train_test["test"]
}


In [ ]:
tinishakespeare

In [ ]:
wikitext2 = load_dataset("Salesforce/wikitext", "wikitext-2-raw-v1")

In [ ]:
wikitext2["train"][0:100]

# 2. Tokenización y limpieza

In [ ]:
dataset = tinishakespeare["train"]
dataset["text"]

In [ ]:
'''# Limpieza en paralelo
def parallel_process_text(data, cleaning_function, num_workers):
    with Pool(num_workers) as pool:
        cleaned_data = pool.map(cleaning_function, data)
    return cleaned_data'''

In [ ]:
'''# Elimina etiquetas HTML y caracteres especiales
def remove_html_tags(text):
    clean_text = re.sub(r'<.*?>', '', text)
    return clean_text

def remove_special_characters(text):
    clean_text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return clean_text

# Pasa a minúsculas
def convert_to_lowercase(text):
    lowercased_text = text.lower()
    return lowercased_text

# Elimina stop words en inglés
def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return filtered_tokens

# Elimina duplicados
def remove_duplicates(texts):
    unique_texts = list(set(texts))
    return unique_texts

# Arregla problemas de encoding
def fix_encoding(text):
    try:
        text = ftfy.fix_text(text)
        decoded_text = text.encode('utf-8').decode('utf-8')
        
    except UnicodeDecodeError:
        decoded_text = 'Encoding Error'
    return decoded_text

# Elimina varios espacios en blanco
def remove_whitespace(text):
    cleaned_text = ' '.join(text.split())
    return cleaned_text'''

In [ ]:
'''from nltk.tokenize import word_tokenize

def tokenize_text(text):
    tokens = word_tokenize(text)
    return tokens'''

In [ ]:
'''
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

def stem_text(tokens):
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    return stemmed_tokens

def lemmatize_text(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return lemmatized_tokens'''

In [ ]:
'''
# Corrige palabras mas escritas. No se aplica pues el texto ya está limpio y  puede corregir de más
def correct_spelling(text):
    spell = SpellChecker()
    tokens = word_tokenize(text)
    corrected_tokens = [spell.correction(word) for word in tokens]
    corrected_text = ' '.join(corrected_tokens)
    return corrected_text'''

In [ ]:
'''from langdetect import detect

def detect_language(text):
    try:
        language = detect(text)
    except:
        language = 'unknown'
    return language'''

In [13]:

def light_clean_fn(example):
    t = example["text"]
    t = ftfy.fix_text(t)
    t = unicodedata.normalize("NFKC", t)
    t = (t.replace("“", '"').replace("”", '"').replace("’", "'")
           .replace("–", "-").replace("—", "-"))
    t = re.sub(r"[ \t]+", " ", t)
    t = re.sub(r"\s*\n\s*", "\n", t)
    t = t.strip(" \n")
    return {"text": t}


In [14]:
from datasets import DatasetDict
cleaned = DatasetDict({
    'train': tinishakespeare['train'].map(light_clean_fn, num_proc=4),
    'validation': tinishakespeare['validation'].map(light_clean_fn, num_proc=4),
    'test': tinishakespeare['test'].map(light_clean_fn, num_proc=4),
})


In [ ]:
cleaned["train"]["text"]

In [16]:
# Escribimos SOLO el train a un archivo para entrenar SP

train_path = Path("./datasets/shakespeare_clean_train.txt")
with train_path.open("w", encoding="utf-8") as f:
    for line in cleaned["train"]["text"]:
        if line.strip():
            f.write(line.strip() + "\n")


In [ ]:
import sentencepiece as spm

spm.SentencePieceTrainer.Train(
    input="./datasets/shakespeare_clean_train.txt", # Corpus de train, se pueden pasar varios
    model_prefix="bpe_model_shakespeare",        # generaprefijo modelos generados:  bpe_model_shakespeare.model y bpe_model_shakespeare.vocab
    vocab_size=16000,                 # 8k–32k para corpora pequeños, 32 k para wikitext2
    model_type="bpe",
    character_coverage=1.0,           # inglés
    byte_fallback=True,               # evita UNK en chars raros
    normalization_rule_name="nfkc",  # Normalización previa
    remove_extra_whitespaces=True, # colapsa espacios extra
    unk_id=0, bos_id=1, eos_id=2, pad_id=3 # <bos> es el inicio de una secuencia (-1 se deshabilita), <eos> el final, <unk> desconocido, <pad> padding
)


In [18]:
import sentencepiece as spm
import os
if os.path.exists("bpe_model_shakespeare.model"):
    print("Existee")


OSError: Not found: "bpe_model_shakespeare.model": No such file or directory Error #2

In [ ]:
import sentencepiece as spm
sp = spm.SentencePieceProcessor(model_file="bpe_model_shakespeare.model")

def sp_encode_batch_train(batch):
    # BPE-dropout: sampling activado
    ids = [
        [sp.bos_id()] +
        sp.encode(t, out_type=int, enable_sampling=True, nbest_size=-1, alpha=0.1) +
        [sp.eos_id()]
        for t in batch["text"]
    ]
    attn = [[1]*len(x) for x in ids]
    return {"input_ids": ids, "attention_mask": attn}

def sp_encode_batch_eval(batch):
    # determinista para val/test
    ids = [
        [sp.bos_id()] + sp.encode(t, out_type=int) + [sp.eos_id()]
        for t in batch["text"]
    ]
    attn = [[1]*len(x) for x in ids]
    return {"input_ids": ids, "attention_mask": attn}

tokenized_train = cleaned["train"].map(sp_encode_batch_train, batched=True, num_proc=4, remove_columns=["text"])
tokenized_val   = cleaned["validation"].map(sp_encode_batch_eval, batched=True, num_proc=4, remove_columns=["text"])
tokenized_test  = cleaned["test"].map(sp_encode_batch_eval, batched=True, num_proc=4, remove_columns=["text"])


In [ ]:
# https://github.com/google/sentencepiece
sp = spm.SentencePieceProcessor(model_file="bpe_model.model")

def sp_encode_batch(batch, add_bos_eos=True):
    if add_bos_eos:
        ids = [[sp.bos_id()] + sp.encode(t, out_type=int) + [sp.eos_id()] for t in batch["text"]]
    else:
        ids = [sp.encode(t, out_type=int) for t in batch["text"]]
    attn = [[1]*len(x) for x in ids]
    return {"input_ids": ids, "attention_mask": attn}

tokenized = cleaned.map(
    sp_encode_batch,
    batched=True,
    num_proc=4,
    remove_columns=["text"]
)

print(tokenized)